# AWS Glue Studio Notebook
##### You are now running a AWS Glue Studio notebook; To start using your notebook you need to start an AWS Glue Interactive Session.


#### Optional: Run this cell to see available notebook commands ("magics").


In [ ]:
%help

####  Run this cell to set up and start your interactive session.


In [5]:
%idle_timeout 60
%glue_version 4.0
%worker_type G.1X
%number_of_workers 2

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 02cfb0b6-4c27-432b-be2d-c4793107980a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 60 minutes.
idle_timeout has been set to 60 minutes.


You are already connected to a glueetl session 02cfb0b6-4c27-432b-be2d-c4793107980a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


You are already connected to a glueetl session 02cfb0b6-4c27-432b-be2d-c4793107980a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 02cfb0b6-4c27-432b-be2d-c4793107980a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 2
Setting new number of workers to: 2



In [6]:
from datetime import *
current_date = datetime.today().strftime('%Y-%m-%d')

s3_path = f's3://video-transient-bucket/{current_date}/'

dynamic_frame = glueContext.create_dynamic_frame.from_options(
    connection_type="s3",
    connection_options={"paths": [s3_path]},
    format="csv",
    format_options={"withHeader": True}#,"optimizePerformance": True}
)
dynamic_frame.printSchema()
dynamic_frame.count()

root
|-- DateTime: string
|-- VideoTitle: string
|-- events: string
|-- id: string

995676


In [7]:
df = dynamic_frame.toDF()

if df.columns != ['DateTime','VideoTitle','events','id']:
    spark.stop()
    job.commit()
    print("The dataset is lack of necessary columns")
    sys.exit(0)

In [9]:
# import boto3
# glue_client = boto3.client('glue', region_name='ap-southeast-2')

# # 检查并创建数据库
# database_name = 'video_raw_data'
# try:
#     response = glue_client.get_database(Name=database_name)
#     print(f"Database {database_name} already exists.")
# except glue_client.exceptions.EntityNotFoundException:
#     response = glue_client.create_database(
#         DatabaseInput={
#             'Name': database_name,
#             'Description': 'Database for video raw data'
#         }
#     )
#     print(f"Database {database_name} created.")
    
# from datetime import *
# current_date = datetime.today().strftime('%Y-%m-%d')
# s3output = glueContext.getSink(
#   path=f"s3://video-raw-bucket/{current_date}",
#   connection_type="s3",
#   updateBehavior="UPDATE_IN_DATABASE",
#   partitionKeys=[],
#   compression="snappy",
#   enableUpdateCatalog=True,
#   transformation_ctx="s3output",
# )
# s3output.setCatalogInfo(
#   catalogDatabase="video_raw_data", catalogTableName="video"
# )
# s3output.setFormat("glueparquet")
# s3output.writeFrame(dynamic_frame)

Database video_raw_data created.


In [8]:
from datetime import *
current_date = datetime.today().strftime('%Y-%m-%d')
output_path = f's3://video-raw-bucket/{current_date}'
glueContext.write_dynamic_frame.from_options(
    frame=dynamic_frame, 
    connection_type="s3", 
    connection_options={"path": output_path},
    format="csv"
)
job.commit()


In [9]:
# # 读取参数
# input_path = output_path

# # 读取 S3 上的 CSV 文件为 DynamicFrame
# dynamic_frame = glueContext.create_dynamic_frame.from_options(
#     connection_type="s3",
#     connection_options={"paths": [input_path]},
#     format="csv",
#     format_options={"withHeader": True,
#                    "optimizePerformance": True}
# )
# print(dynamic_frame.count())

995676
